# DW_oa_Task04
## python自动化之PDF
## 20210620

### Python 操作 PDF

PDF 操作是本次自动化办公的最后一个知识点，初级的 PDF 自动化包括 PDF 文档的拆分、合并、提取等操作，更高级的还包括 WORD与PDF互转等

初级操作一般比较常用，也可以解决较多的办公内容，所以本节将会主要介绍 PDF 的初级操作，具体内容将会从以下几个小节展开：

1. 相关介绍
2. 批量拆分
3. 批量合并
4. 提取文字内容
5. 提取表格内容
6. 提取图片内容
7. 转换为PDF图片
8. 添加水印
9. 加密与解码

下面直接开始本节内容。

#### 1. 相关介绍

Python 操作 PDF 会用到两个库，分别是：PyPDF2 和 pdfplumber

其中 **PyPDF2** 可以更好的读取、写入、分割、合并PDF文件，而 **pdfplumber** 可以更好的读取 PDF 文件中内容和提取 PDF 中的表格

对应的官网分别是：

> PyPDF2：https://pythonhosted.org/PyPDF2/
>
> pdfplumber：https://github.com/jsvine/pdfplumber

由于这两个库都不是 Python 的标准库，所以在使用之前都需要单独安装

win+r 后输入 cmd 打开 command 窗口，依次输入如下命令进行安装：

In [1]:
!pip install pypdf2

In [2]:
!pip install pdfplumber

Processing c:\users\jo\appdata\local\pip\cache\wheels\f2\b1\a0\c0a77b756d580f53b3806ae0e0b3ec945a8d05fca1d6e10cc1\pdfplumber-0.5.28-py3-none-any.whl
  Using cached pdfminer.six-20200517-py3-none-any.whl (5.6 MB)
  Using cached pycryptodome-3.10.1-cp35-abi3-win_amd64.whl (1.6 MB)


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] 拒绝访问。: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\Crypto\\__init__.py'
Consider using the `--user` option or check the permissions.



#### 2. 批量拆分

将一个完整的 PDF 拆分成几个小的 PDF，因为主要涉及到 PDF 整体的操作，所以本小节需要用到 PyPDF2 这个库

拆分的大概思路如下：

- 读取 PDF 的整体信息、总页数等
- 遍历每一页内容，以每个 step 为间隔将 PDF 存成每一个小的文件块
- 将小的文件块重新保存为新的 PDF 文件

需要注意的是，在拆分的过程中，可以手动设置间隔，例如：每5页保存成一个小的 PDF 文件

拆分的代码如下：

In [3]:
def split_pdf(filename, filepath, save_dirpath, step=5):
    """
    拆分PDF为多个小的PDF文件，
    @param filename:文件名
    @param filepath:文件路径
    @param save_dirpath:保存小的PDF的文件路径
    @param step: 每step间隔的页面生成一个文件，例如step=5，表示0-4页、5-9页...为一个文件
    @return:
    """
    if not os.path.exists(save_dirpath):
        os.mkdir(save_dirpath)   # 没有保存的文件路径,则新建
    pdf_reader = PdfFileReader(filepath)
    # 读取每一页的数据
    pages = pdf_reader.getNumPages()
    for page in range(0, pages, step):
        pdf_writer = PdfFileWriter()
        # 拆分pdf，每 step 页的拆分为一个文件
        for index in range(page, page+step):
            if index < pages:
                pdf_writer.addPage(pdf_reader.getPage(index))
        # 保存拆分后的小文件
        save_path = os.path.join(save_dirpath, filename+str(int(page/step)+1)+'.pdf')
        print(save_path)
        with open(save_path, "wb") as out:
            pdf_writer.write(out)

    print("文件已成功拆分，保存路径为："+save_dirpath)

In [57]:
def split_pdf1(filename, filepath, save_dirpath, step=5):
    """
    拆分PDF为多个小的PDF文件，
    @param filename:文件名
    @param filepath:文件路径
    @param save_dirpath:保存小的PDF的文件路径
    @param step: 每step间隔的页面生成一个文件，例如step=5，表示0-4页、5-9页...为一个文件
    @return:
    """
    if not os.path.exists(save_dirpath):
        os.mkdir(save_dirpath)   # 没有保存的文件路径,则新建
    pdf_reader = PdfFileReader(filepath)
    # 读取每一页的数据
    pages = pdf_reader.getNumPages()
    print(pages)
    n = 0
    for page in range(0, pages):
        pdf_writer = PdfFileWriter()
        # 拆分pdf，每 step 页的拆分为一个文件
        pdf_writer.addPage(pdf_reader.getPage(page))
        fy = n+step
        print(page)
        if fy > pages:
            fy = pages
        if page == fy-1:
            n = fy
            pdf_writer.addPage(pdf_reader.getPage(fy))
        # 保存拆分后的小文件
        
        save_path = os.path.join(save_dirpath, filename+str(int(page/step)+1)+'.pdf')
        print(save_path)
        with open(save_path, "wb") as out:
            pdf_writer.write(out)

    print("文件已成功拆分，保存路径为："+save_dirpath)

In [5]:
import os
from PyPDF2 import PdfFileReader, PdfFileWriter
split_pdf('good','D:\\pythontest\\DW_oa\\tempfile.pdf','D:\\pythontest\\DW_oa\\temp',step=3)

D:\pythontest\DW_oa\temp\good1.pdf
D:\pythontest\DW_oa\temp\good2.pdf
D:\pythontest\DW_oa\temp\good3.pdf
文件已成功拆分，保存路径为：D:\pythontest\DW_oa\temp


In [58]:
split_pdf1('cool','D:\\pythontest\\DW_oa\\tempfile.pdf','D:\\pythontest\\DW_oa\\temp1',step=3)

8
0
D:\pythontest\DW_oa\temp1\cool1.pdf
1
D:\pythontest\DW_oa\temp1\cool1.pdf
2
D:\pythontest\DW_oa\temp1\cool1.pdf
3
D:\pythontest\DW_oa\temp1\cool2.pdf
4
D:\pythontest\DW_oa\temp1\cool2.pdf
5
D:\pythontest\DW_oa\temp1\cool2.pdf
6
D:\pythontest\DW_oa\temp1\cool3.pdf
7


IndexError: list index out of range

用自己的思维重写了一下,发现运行结果有问题,是思维逻辑的原因,理不出来,回头再来看看

**需要注意的是：**

如果你是第一次运行代码，在运行过程中，会直接报如下的错误

![](https://raw.githubusercontent.com/double-point/GraphBed/master/python_2_pdf/%E6%8B%86%E5%88%86%E6%8A%A5%E9%94%99.png)

如果是在 Pycharm 下，直接通过报错信息，点击 utils.py 文件，定位到第 238 行原文

原文中是这样的：

```python
 r = s.encode('latin-1')
 if len(s) < 2:
   		bc[s] = r
 return r
```

修改为：

```python
try:
    r = s.encode('latin-1')
    if len(s) < 2:
        bc[s] = r
    return r
except Exception as e:
    r = s.encode('utf-8')
    if len(s) < 2:
        bc[s] = r
    return r
```

如果你使用的是 **anaconda**，对应的文件路径应该为：anaconda\Lib\site-packages\PyPDF2\utils.py，进行同样的修改操作即可


#### 3. 批量合并

比起拆分来，合并的思路更加简单：

- 确定要合并的 **文件顺序**
- 循环追加到一个文件块中
- 保存成一个新的文件

对应的代码比较简单，基本不会出现问题：

In [27]:
def concat_pdf(filename, read_dirpath, save_filepath):
    """
    合并多个PDF文件
    @param filename:文件名
    @param read_dirpath:要合并的PDF目录
    @param save_filepath:合并后的PDF文件路径
    @return:
    """
    pdf_writer = PdfFileWriter()
    # 对文件名进行排序
    list_filename = os.listdir(read_dirpath)
    list_filename.sort(key=lambda x: int(x[:-4].replace(filename, "")))
    for filename in list_filename:
        print(filename)
        filepath = os.path.join(read_dirpath, filename)
        # 读取文件并获取文件的页数
        pdf_reader = PdfFileReader(filepath)
        pages = pdf_reader.getNumPages()
        # 逐页添加
        for page in range(pages):
            pdf_writer.addPage(pdf_reader.getPage(page))
    # 保存合并后的文件
    with open(save_filepath, "wb") as out:
        pdf_writer.write(out)
    print("文件已成功合并，保存路径为："+save_filepath)

In [34]:
concat_pdf('good', 'D:\\pythontest\\DW_oa\\temp', 'D:\\pythontest\\DW_oa\\good.pdf')

good1.pdf
good2.pdf
good3.pdf
文件已成功合并，保存路径为：D:\pythontest\DW_oa\good.pdf
